<a href="https://colab.research.google.com/github/theinshort/crawler/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#WEB DATA CRAWLING

In this project our aim is to scrap a webiste using python.

`This project is for learning purpose and is not intended to perform any kind of violations.`

## Objective
We will use a web crawler to acquire data from a specific domain

1. Choose a domains of interest (e.g., news articles, product reviews, scientific publications etc).
2. Identify and use web crawling tools or libraries (such as BeautifulSoup, Scrapy, or others) to extract data from the chosen domains.
3. Collect a sufficient amount of data to ensure diversity and relevance.
4. Scrape and clean the HTML contents to generate clean text outputs (at least 2 GB textual data, the more than better).

## Final Outcome


1. Colab Notebook:
- Showcases the entire process of web data crawling, including the
chosen domains, code implementation, and data extraction.
- Clearly comment and document each step in the notebook.
2. Dataset Files:
- Extracted dataset in a separate file format (e.g., CSV, JSON) that includes a sample of the collected data.
3. Summary:
- Why specific domains were selected.
- Briefly describe the web crawling tools or libraries used and why?
- Statistics of data extracted from each domain.